In [ ]:
# !pip install ckiptagger

In [418]:
# from ckiptagger import data_utils
# data_utils.download_data_gdown("./")

In [373]:
import pandas as pd
tbl_post = pd.read_csv('tbl_post.csv')

In [12]:
# 切詞用
def strQ2B(ustring):
    """把字串全形轉半形"""
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全形空格直接轉換
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全形字元（除空格）根據關係轉化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return ''.join(ss)

# 停用詞
with open('stopwords_tc.txt', encoding='utf-8', mode='r') as f:
    stop_words = []
    for l in f:
        stop_words.append(l.strip())


In [403]:
aaa = aaa.append(product_id_index_1066)
aaa = aaa.reset_index(drop=True)
aaa

,product_id,content_cut
0,1,"[三月, 中, 買, SE3, 昨天, 發生, 次, 自動, 重啟, 手機, 電, 螢幕, ..."
1,2,"[['Engadget , 編輯, 團隊, 致力, 蒐羅, 優質, 產品, 優惠, 價格, ..."
2,3,"[['Engadget , 編輯, 團隊, 致力, 蒐羅, 優質, 產品, 優惠, 價格, ..."
3,4,"[一大早, 黑, 貓, 送貨, '', ', 收到, ['[, 簡評, ] iPhone..."
4,5,"[['Engadget , 編輯, 團隊, 致力, 蒐羅, 優質, 產品, 優惠, 價格, ..."
...,...,...
69,957,"[['POCO , 新, 電競, 旗艦機, POCO F4, GT , 囉, 場, 酣暢..."
70,958,"[Content, :───────────────────────────────────..."
71,959,"[請問, Redmi , 10, C, 投影, 電視, 感謝, 小弟, 最近, 買, 隻, ..."
72,1066,"[高階, 手機, 主相機, 搭載, 越來越, 感光, 元件, SHARP AQUOS R6,..."


In [416]:
aaa.to_csv('Content_cut.csv', index=False, encoding="utf-8")

In [406]:
# 切詞
from ckiptagger import WS, POS, NER
import pandas as pd
# max(tbl_post['product_id'].unique())==1067-product_id最大
max_product_id = max(tbl_post['product_id'].unique())

ws = WS("./data")
Content_cut = []

for i in range(max_product_id+1):
    # 同產品->product
    product1 = tbl_post[tbl_post['product_id']==i]
    product = product1.reset_index(drop=True)
    per_word_cut = []
    
    for j in range(len(product)):
        ws_results = ws([product['Content'][j]])
        for word in ws_results[0]:
            word = strQ2B(word)
            if word in stop_words:
                continue
            elif (word==" " or word=="「" or word=="」" or word=="['" or word=="']"):
                continue
            else:
                per_word_cut.append(word)
    Content_cut.append([per_word_cut])
    
Content_cutting = pd.DataFrame(Content_cut)
Content_cutting

In [417]:
# 處理
product_id_Index=[]
max_product_id = max(tbl_post['product_id'].unique())
for i in range(max_product_id):
    i = i+1
    product_id_Index.append(i)
product_id_Index = pd.DataFrame(product_id_Index)
product_id_Index['content_cut'] = Content_cut[0]
product_id_Index.rename(columns = {0:'product_id'}, inplace = True)
# 將該id沒有爬蟲內容者過濾
product_id_Index_final = product_id_Index[product_id_Index.content_cut.str.len()>0].reset_index(drop=True)
product_id_Index_final.to_csv('Content_cut.csv', index=False, encoding="utf-8")